<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST2312_H11/blob/main/CST2312_Assignment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CST2312 Assignment 05**

*by Professor Patrick, created 27-Sep-2021 | updated: 04-Feb-2023, 25-Feb-2025, 19-Sep-2025, 26-Jan-2026*       

This notebook includes scaffolding for a three-part  assignment using APIs.


## Exercises: Using Parameters with API Calls



The first API call that we tried was very simple. We just fetched a URL. Now let's see a URL that accepts as input a set of **parameters**. We have already seen this concept with functions; the parameters of the API calls are the exact equivalent but for Web APIs, which are, at their core, functions that we call over the web.

### Example: OpenWeatherMap

Let's try to query OpenWeatherMap now, to get data about the weather. [Documentation](http://openweathermap.org/current#geo). Below you can find the URL that you can copy and paste in your browser, to get the weather for New York. You will notice that it contains parameters as part of the URL, including an `appid` which is a key that is used to limit the number of calls that can be issued by a single application.

Try the URL in your browser. Also try to change the query parameter `q` and change it from `New%20York,NY` to something different. (Note: The `%20` is a transformation for the space (` `) character in URLs.)

<b>You will need to get an API key from [`openweathermap`](https://openweathermap.org/price)</b>  


#### 🔐 Prerequisites

To use the `openweathermap.org` API, you'll need:

1. A **free or paid API key** from [http://openweathermap.org](http://openweathermap.org).
2. Basic knowledge of making HTTP requests in Python.
3. Store your API key value as `openweathermapAPIkey` in the Colab Secrets tab.

See the *APPENDIX: Using Veracrypt for Encrypted Storage* at the bottom of this notebook for instructions on using the free `Veracrypt.fr` application to create an encrypted drive where you can store your API keys.  


### 📦 Required Libraries

We will use the `requests` library to interact with the API. If it's not already installed, run:

```python
!pip install requests
```

Once you have an `openweathermap` account, you can check your API keys with this [dashboard](https://home.openweathermap.org/api_keys)  

The following code uses a Python dictionary to organize and list the parameters.

In [ ]:
import requests
from google.colab import userdata   # using the Google Colab `secrets` module
api_key = userdata.get('openweathermapAPIkey')  # getting the value for `ipstackAPIkey` from `secrets`


openweathermap_url = "http://api.openweathermap.org/data/2.5/weather"
parameters = {
    'q'     : 'New York, NY, USA',
    'units' : 'metric',
    'mode'  : 'json',
    'appid' : f'{api_key}'
}
resp = requests.get(openweathermap_url, params=parameters)
data = resp.json()
print(data)

In [ ]:
openweathermap_url = "http://api.openweathermap.org/data/2.5/weather"
parameters = {
    'q'     : 'New York, NY, USA',
    'units' : 'metric',
    'mode'  : 'json',
    'appid' : f'{api_key}'
}
resp = requests.get(openweathermap_url, params=parameters)
data = resp.json()
print(data['main'])


In [ ]:
resp = requests.get("http://api.openweathermap.org/data/2.5/weather?q=New%20York,NY,USA&units=imperial&mode=json&appid="+api_key)
data = resp.json()
print(data)



---



### Example: Visual Crossing  


###Weather Request with **HTTP** `GET`    

*This example queries the server at `visualcrossing.com` with some arguments describing the weather data requested then processes the response as either a <b>CSV</b> file or <b>JSON</b>.*

####Housekeeping    

<u>**Prerequisites**</u>    

Before starting, you need access to Weather API we are going to use. If you don’t have account already, you can sign up for a free account at [Visual Crossing Weather Services](https://www.visualcrossing.com).     

*Visual Crossing offers a two-month free tier of up to 1000 results per day. if you need more, you can pay-per-result or sign up for a monthly plan.*

The Python scripts in this section have been written in Python 3.8.2 but should work in most recent versions.    

In this sample we are going to use the `CSV` format to download the data so we include a library to help process the data that is returned by the API.

Here are the import statements for this example...   


In [ ]:
# import the csv library to write output to a `.csv` file
import csv

# import the codecs library for string encoding and decoding
import codecs

# import `.request()` and `.error()` from the urllib library
import urllib.request
import urllib.error

# import the sys library for operating system instructions - BE CAREFUL WITH THIS
import sys

# store the API key for VisualCrossing from Colab Secrets
from google.colab import userdata
api_key = userdata.get('visualcrossingAPIkey')

####Setting up the weather data input parameters    


The first part of the exercise sets up the parameters to download the weather data and the second part retrieves the weather data.     

The weather data is retrieved using a **REST**ful weather **API** so we simply have to create a web query within the Python script and download the data.    

The first part of the sets up some parameters to customize the weather data that is retrieved.      


In [ ]:
## 0
# Assign the API URL to a variable named `BaseURL`
BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

## 1
# api_key='YOUR_API_KEY'
#  - or -
api_key = api_key  # <-- API_key? (from Colab Secrets variable visualcrossingAPIkey)

## 2
# UnitGroup sets the units of the output - us or metric
UnitGroup='us'

## 3
# Location for the weather data
Location='Washington,DC'

## 4
# Optional start and end dates
# If nothing is specified, the forecast is retrieved.
# If start date only is specified, a single historical or forecast day will be retrieved
# If both start and and end date are specified, a date range will be retrieved
# Use ISO format for dates - see: https://www.iso.org/iso-8601-date-and-time-format.html
StartDate = ''
EndDate=''

## 5
# JSON or CSV
# JSON format supports daily, hourly, current conditions, weather alerts and events in a single JSON package
# CSV format requires an 'include' parameter below to indicate which table section is required
ContentType="csv"

## 6
# include sections or periodicity and other Visual Crossing components
# values include days,hours,current,alerts
Include="days"

In the above code, we set up parameters. for the script. To keep this script simple, we’re hardcoded variables for the various parameters to help with readability.    

 1. The first parameter is the **API** Key which is provided when signing up for the **API**. You can access it from the [‘Account’ location within Weather Data Account Page](https://www.visualcrossing.com/account).  

 2. The second parameter is the unit of measure to be used in the result.  This **API** wants to refer to either `metric` or `us` measures where `us` refers to *imperial* measures such as degrees in Fahrenheit, miles, feet, inches, etc.  The value for this parameter is stored in the string variable `UnitGroup`.     

 3. The third parameter is the location for which the weather data should be downloaded. This is in form of an address, partial address or latitude and longitude value (for example 35.46,-75.12).  The value for this paramater is stored in the string variable `Location`.       

 4. Next, we can specify the date range of information we are interested in. The Weather **API** will automatically request historical or forecast data based on the date range requested. The code requests a start and end date in the form YYYY-MM-DD, for example 2020-03-26 is the 26th March, 2020. The format of the date is important for the weather **API** query.  The values for these parameters are stored in the string variables `StartDate` and `EndDate`, respectively.       

    - If start date isn’t specified, the query will request the next 15 days weather forecast.    

    - You can also use a dynamic date period as the start date such as yesterday, tomorrow, last7 days etc. See [Weather Data Periods](https://www.visualcrossing.com/resources/documentation/weather-api/using-the-time-period-parameter-to-specify-dynamic-dates-for-weather-api-requests/).    

 5. We then use the string variable `ContentType` to specify the format for the  content returned.   

 6. Finally, in this example, we specify the periodicity of the content to be returned in the string variable `Include`.    

Many more **API** parameters are available. For more information on the full set of Weather **API** parameters, see the [Weather **API** documentation](https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/).    

####Downloading the weather data    

The next section of code creates the Weather **API** request from the parameters, submits the request to the server and then parses the result.     



In [ ]:
#basic query including location
ApiQuery=BaseURL + Location

#append the start and end date if present
if (len(StartDate)):
    ApiQuery+="/"+StartDate
    if (len(EndDate)):
        ApiQuery+="/"+EndDate

#Url is completed. Now add query parameters (could be passed as GET or POST)
ApiQuery+="?"

#append each parameter as necessary
if (len(UnitGroup)):
    ApiQuery+="&unitGroup="+UnitGroup

if (len(ContentType)):
    ApiQuery+="&contentType="+ContentType

if (len(Include)):
    ApiQuery+="&include="+Include

ApiQuery+="&key="+api_key


In [ ]:
print('The constructed API query is:', ApiQuery)

The first part of the code constructs the requests to form a single URL. In this example, we are sending the request as a **GET** request.     

Other request techniques are available such as **POST**, which is useful if your list of locations is long or even [**ODATA**](https://www.visualcrossing.com/resources/documentation/weather-api/how-do-i-retrieve-weather-data-using-an-odata-request/) for specialized data import and data science applications.     

See the [Weather API documentation](https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/) section for more information.

In [ ]:
print(' - Running query URL: ', ApiQuery)
print()

try:
    CSVBytes = urllib.request.urlopen(ApiQuery)
except urllib.error.HTTPError  as e:
    ErrorInfo= e.read().decode()
    print('Error code: ', e.code, ErrorInfo)
    sys.exit()


The final two lines of the code download the requested weather data and provides some simple error handling. We have used the `urllib.request` library to provide the retrieval functionality here.

<u>**Error handling with `urllib.request`**</u>    

It is important to provide error handling so that problems may be resolved quickly. When providing the error handling, ensure that you check the HTTP response code and also read the response body.    

The response body will contain full details of the Weather API error and is the best way to resolve most problems. Another useful way to troubleshoot API requests that are returning an error is to copy the URL from the code into a browser window. This will provide a quick and easy way to see any errors that are being returned. This technique can also be used to see the structure of the weather data. Don’t forget you can also use the Weather Data Services query builder page to construct requests and see results.    


<u>**Using the weather data**</u>    

As mentioned above, we are using Comma Separated Values for the output in this example, but JSON format is available too. The **CSV** data is encoded in **UTF-8** encoding so we indicate that to ensure accurate decoding.    


In [ ]:
import csv

import codecs

# Parse the results as CSV
CSVText = csv.reader(codecs.iterdecode(CSVBytes, 'utf-8'))

In [ ]:
print(CSVText)


We now have the weather data as a CSVText instance.     

From here we can use the data in many ways.     
For example we can [analyze the weather data in R](https://www.visualcrossing.com/resources/documentation/weather-data-tutorials/how-can-i-find-historical-weather-data-for-analysis-in-r/), [load it into a database](https://www.visualcrossing.com/resources/blog/how-to-import-weather-data-into-mysql/) or simply display it to the user. In this example, we simply display the user to the screen.

*The raw data is simply a table of weather data rows. In this case the historical weather data for each day requested.*

```
name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
"Washington, DC, United States",2021-12-14,58.9,34.8,44.3,58.9,34.8,44,28.4,58.5,0,0,,,0,0,9.2,4.7,66.6,1032.2,34.4,12.5,118,7.1,4,10,2021-12-14T07:19:08,2021-12-14T16:46:50,0.41,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KDCA,F0198,KADW,KDAA,PWDM2"
"Washington, DC, United States",2021-12-15,54.1,36,45.3,54.1,36,45.1,39,78.9,0,4,,,0,0,8.5,4.3,109.6,1031.5,63.5,15,81.4,7.1,4,10,2021-12-15T07:19:49,2021-12-15T16:47:07,0.44,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,
"Washington, DC, United States",2021-12-16,62,45,53.7,62,41.9,52.6,36.1,67.1,0,16,,,0,0,20.8,10.3,208.8,1019.9,55.7,15,73.3,6.4,3,10,2021-12-16T07:20:29,2021-12-16T16:47:26,0.47,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,Address,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,Wind Direction,Wind Chill,Precipitation,Precipitation Cover,Snow Depth,Visibility,Cloud Cover,Sea Level Pressure,Weather Type,Latitude,Longitude,Resolved Address,Name,Info,Conditions
"Herndon,VA",01/01/2019,44.6,60.8,53.1,45.1,75.58,,23.2,36.3,269.29,41.4,0,8.33,,9.1,83.5,1014.8,"Mist, Fog, Light Rain",38.96972,-77.38519,"Herndon,VA","","","Overcast"
"Herndon,VA",01/02/2019,37.3,44.9,42.4,35.9,78.17,,8.2,,143.95,38.7,0,0,,10,98.3,1023.6,"",38.96972,-77.38519,"Herndon,VA","","","Overcast"

```


The following code simply steps through the rows of **CSV** data and prints out the data.


In [ ]:
RowIndex = 0

# The first row contain the headers and the additional rows each contain the weather metrics for a single day
# To simply our code, we use the knowledge that column 0 contains the location and column 1 contains the date.  The data starts at column 4
for Row in CSVText:
    if RowIndex == 0:
        FirstRow = Row
    else:
        print('Weather in ', Row[0], ' on ', Row[1])

        ColIndex = 0
        for Col in Row:
            if ColIndex >= 4:
                print('   ', FirstRow[ColIndex], ' = ', Row[ColIndex])
            ColIndex += 1
    RowIndex += 1


---

## Assignment 05.1


a. Extract the current temperature from the returned JSON response.


In [ ]:
# your code here
data['main']['feels_like']

b. Extract the description of the current weather

In [ ]:
# your code here
data['weather'][0]['description']

c. Try to change the units to `metric` and repeat


In [ ]:
# your code here

d. Get the weather for San Francisco, CA



In [ ]:
# your code here


## Assignment 05.2



* Study the documentation of the API ([Documentation](http://openweathermap.org/current#geo)). Change the API call to use the longitute and latitude.

In [ ]:
# your code here


## Assignment 05.3



Read the location of your computer using the GeoIP API. Then use the OpenWeatherMap to query the API and fetch the temperature for the location returned by the GeoIP `ipstack.com` API. For this exercise, you will need to learn to read variables from a Web API (the `ipstack` GeoIP) and use them as input in another (`openweathermap`)

In [ ]:
# your code here




---



> *Submit your solutions to the assignment on Brightspace.  Please download your entire Colab notebook and submit the file downloaded.  The assignment on Brightspace will have more details.*



---

